# Using SVD to do Dimensionality Reduction on MNIST Dataset by `Mr. Harshit Dawar!`

* **Note: Using normal SVD form either Numpy or Scipy, one can't mention the no. of features to select, whereas, using truncated SVD, gives you the authority to mention the no. of features to select.**


* **Moreover, SVD use-cases are data compression & dimensionality reduction, whereas PCA can be used for dimensionality reduction, & the situations where the variance is most concerned thing. PCA internally uses SVD to decompose the covariance matrix instead of performing EigenDecomposition because SVD is much more efficient than the normal Eigen Decomposition as it can handle sparse matrixes (non-squre matrices).**

## Importing the required Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.datasets import load_digits
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split